In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import gensim

from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

# Any results you write to the current directory are saved as output.

['ks-projects-201801.csv', 'ks-projects-201612.csv']


In [2]:
df = pd.read_csv('../input/ks-projects-201801.csv')
df.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


In [ ]:
names_data = df.loc[:,'name']
names_data.head()

In [29]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))

documents = []
un_documents = []
for index, row in df.iterrows():
    if row['state'] == 'successful':
        un_documents.append(read_questions(row,"name"))
    if row["state"] == 'failed':
        documents.append(read_questions(row,"name"))

print(type(documents))
print(type(un_documents))

<class 'list'>
<class 'list'>


In [3]:
#simplified_df = df.transform()
idx = np.logical_or(df['state']=='successful',df['state']=='failed')
docs = df.loc[idx,'name'].values.astype('U')
feature_2 = df.loc[idx,'main_category'].values.astype('U')
feature_3 = df.loc[idx,'goal'].values

targets = df.loc[idx,'state'].values

output = np.ones(len(targets),dtype='bool')
for i in range(0,len(targets)):
    if(targets[i]=='failed'):
        output[i] = False

# Shows fraction of inputs that have been successful out of the total dataset.
print(sum(output)/len(targets))

print(feature_3[0]+feature_3[1])

0.4038772895153388
31000.0


In [ ]:
'''print(set(feature_2))
feature_2 = feature_2.reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
print(feature_2.shape)
feature_2 = encoder.fit_transform(feature_2)
print(feature_2.shape)'''

In [4]:
X_train, X_test, y_train, y_test, f2_train, f2_test, f3_train, f3_test = train_test_split(docs, 
                                        output, feature_2, feature_3,test_size = 0.20, random_state = 12)

In [5]:
# Add the argument 'ngram_range' later to see if the efficiency improves with bigrams - no improvements whatsoever with bigrams in case of simple regressors
vect = CountVectorizer(min_df=1)

X_train_counts = vect.fit_transform(X_train)
print(vect.vocabulary_['love'])
feature_transformer = TfidfTransformer()
X_train_tfidf = feature_transformer.fit_transform(X_train_counts)
X_test_counts = vect.transform(X_test)
X_test_tfidf = feature_transformer.transform(X_test_counts)
print(X_test_tfidf.shape)

60497
(66335, 116123)


In [ ]:
# Additional features seemed to not increase the accuracy of prediction by any significant amount
'''print(X_test_tfidf.shape)
print(f2_test.shape)
f3_train = f3_train.reshape(-1,1)
f3_test = f3_test.reshape(-1,1)
print(f3_test.shape)

from scipy import sparse
X_train_n = sparse.hstack((X_train_tfidf,f2_train,f3_train))
X_test_n = sparse.hstack((X_test_tfidf,f2_test,f3_test))
print(X_train_n.shape)
print(X_test_n.shape)
#X_train_n = np.concatenate((X_train_tfidf.toarray(),f2_train,f3_train))
#X_test_n = np.concatenate((X_test_tfidf.toarray(),f2_test,f3_test))'''

In [ ]:

#X_test_n = np.concatenate((X_test_tfidf,f2_test,f3_test))
#X_train, X_test, y_train, y_test, f2_train, f2_test, f3_train, f3_test

In [30]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test_tfidf)
print(accuracy_score(y_test, y_pred))

0.6470038441245195


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lg = LogisticRegression()
lg.fit(X_train_tfidf,y_train)
y_pred = lg.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6541946182256727


In [17]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
lg = LinearSVC(C=0.05)
lg.fit(X_train_tfidf,y_train)
y_pred = lg.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))

0.6558980930127384


In [14]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
br = Ridge(alpha=2)
br.fit(X_train_tfidf,y_train)
y_pred = br.predict(X_test_tfidf)
print(r2_score(y_test,y_pred))

0.0876087597782611


In [28]:
#X_sample = ['adventure debut project play']
X_sample = ['instantly']
X_sample_counts = vect.transform(X_sample)
X_sample_tfidf = feature_transformer.transform(X_sample_counts)
print(X_sample_tfidf.shape)
print(br.predict(X_sample_tfidf))

(1, 116123)
[0.50950736]


In [31]:

model = gensim.models.Word2Vec(size=150, window=10, min_count=2, sg=1, workers=10)
succ_model = gensim.models.Word2Vec(size=150, window=10, min_count=2, sg=1, workers=10)
model.build_vocab(documents)
succ_model.build_vocab(un_documents)
model.train(sentences=documents, total_examples=len(documents), epochs=model.iter)
succ_model.train(sentences=un_documents, total_examples=len(un_documents), epochs=succ_model.iter)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


(3013952, 3694445)

In [32]:
word_vectors = model.wv
count = 0
for word in word_vectors.vocab:
    if count<10:
        print(word)
        count += 1
    else:
        break

the
songs
of
adelaide
greeting
from
earth
arts
capsule
for


In [ ]:
#len(word_vectors.vocab)
#print(model.wv["education"])

In [33]:
tmp = model.wv.most_similar(positive=['mobile'],topn=20)
succ_tmp = succ_model.wv.most_similar(positive=['mobile'],topn=20)

unsucc_list = []
succ_list = []
for i in range(0,len(tmp)):
    unsucc_list.append(tmp[i][0])
    succ_list.append(succ_tmp[i][0])

print(np.setdiff1d(succ_list,unsucc_list))

['ar' 'classroom' 'coding' 'diy' 'electronics' 'energy' 'gardening'
 'hardware' 'hub' 'indoor' 'maker' 'makerspace' 'management' 'network'
 'plate' 'programming' 'store' 'training' 'upgrade' 'vertical']


In [34]:
tmp = model.wv.most_similar(positive=['love'],topn=20)
succ_tmp = succ_model.wv.most_similar(positive=['love'],topn=20)

unsucc_list = []
succ_list = []
for i in range(0,len(tmp)):
    unsucc_list.append(tmp[i][0])
    succ_list.append(succ_tmp[i][0])

print(np.setdiff1d(succ_list,unsucc_list))

['addiction' 'birth' 'equality' 'faith' 'found' 'healing' 'hope' 'illness'
 'jesus' 'loss' 'muslim' 'replacements' 'sexual' 'spread' 'tell' 'times'
 'told']


In [35]:
tmp = model.wv.most_similar(positive=['gaming'],topn=20)
succ_tmp = succ_model.wv.most_similar(positive=['gaming'],topn=20)

unsucc_list = []
succ_list = []
for i in range(0,len(tmp)):
    unsucc_list.append(tmp[i][0])
    succ_list.append(succ_tmp[i][0])

print(np.setdiff1d(succ_list,unsucc_list))

['builder' 'coins' 'dungeon' 'linux' 'miniature' 'module' 'modules'
 'printable' 'rpgs' 'strategy' 'tabletop' 'tactics' 'terrain' 'tile'
 'tiles' 'tokens' 'wargaming']


In [37]:
tmp = model.wv.most_similar(positive=['machine','learning'],topn=20)
succ_tmp = succ_model.wv.most_similar(positive=['machine','learning'],topn=20)

unsucc_list = []
succ_list = []
for i in range(0,len(tmp)):
    unsucc_list.append(tmp[i][0])
    succ_list.append(succ_tmp[i][0])

print(np.setdiff1d(succ_list,unsucc_list))

['ai' 'ar' 'arcade' 'blocks' 'cardboard' 'coding' 'computer' 'content'
 'data' 'developer' 'engine' 'environment' 'programming' 'python' 'racing'
 'speech' 'tutorials' 'vr' 'windows']


In [38]:
# Note -> Compatibility is something not found in the dataset names
# Dating -> Can occur in two relations: Carbon dating & Dating people
tmp = model.wv.most_similar(positive=['dating'],topn=20)
succ_tmp = succ_model.wv.most_similar(positive=['dating'],topn=20)

unsucc_list = []
succ_list = []
for i in range(0,len(tmp)):
    unsucc_list.append(tmp[i][0])
    succ_list.append(succ_tmp[i][0])

print(np.setdiff1d(succ_list,unsucc_list))

['allegiance' 'bigfoot' 'crime' 'destroy' 'erotic' 'fantastical' 'furry'
 'genre' 'guardian' 'humans' 'identity' 'murder' 'otome' 'paranormal'
 'relationships' 'serial' 'superhero' 'ya' 'yaoi' 'zone']


In [39]:
# Note -> Compatibility is something not found in the dataset names
# Dating -> Can occur in two relations: Carbon dating & Dating people
tmp = model.wv.most_similar(positive=['dating','relationships'],topn=20)
succ_tmp = succ_model.wv.most_similar(positive=['dating','relationships'],topn=20)

unsucc_list = []
succ_list = []
for i in range(0,len(tmp)):
    unsucc_list.append(tmp[i][0])
    succ_list.append(succ_tmp[i][0])

print(np.setdiff1d(succ_list,unsucc_list))

['alchemist' 'autistic' 'betrayal' 'bigfoot' 'cops' 'corruption' 'destroy'
 'emotional' 'fantastical' 'friendship' 'furry' 'genre' 'goliath' 'kung'
 'mormon' 'prevention' 'quarry' 'romcom' 'satire' 'serial']
